<a href="https://colab.research.google.com/github/swethakambathula/Global_Wildfire_Spread_AI/blob/main/GlobalWildfireSpreadPredictionCNNLTSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install kaggle


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir -p ~/.kaggle/ && cp /content/drive/MyDrive/kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d fantineh/next-day-wildfire-spread

next-day-wildfire-spread.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/next-day-wildfire-spread.zip

In [ ]:
import re
from typing import Dict, List, Optional, Text, Tuple
import matplotlib.pyplot as plt
from matplotlib import colors
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import io
import imageio
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox
import tensorflow_datasets as tfds

In [ ]:
file_pattern = '/content/next_day_wildfire_spread_train*'

In [ ]:
INPUT_FEATURES = ['elevation', 'th', 'vs',  'tmmn', 'tmmx', 'sph',
                  'pr', 'pdsi', 'NDVI', 'population', 'erc', 'PrevFireMask']

OUTPUT_FEATURES = ['FireMask', ]

DATA_STATS = {'elevation': (0.0, 3141.0, 657.3003, 649.0147), 'pdsi': (-6.1298, 7.8760, -0.0053, 2.6823), 'NDVI': (-9821.0, 9996.0, 5157.625, 2466.6677), 'pr': (0.0, 44.5304, 1.7398051, 4.4828), 'sph': (0., 1., 0.0071658953, 0.0042835088), 'th': (0., 360.0, 190.3298, 72.5985), 'tmmn': (253.15, 298.9489, 281.08768, 8.9824), 'tmmx': (253.15, 315.0923, 295.17383, 9.8155), 'vs': (0.0, 10.0243, 3.8501, 1.4110), 'erc': (0.0, 106.2489, 37.3263, 20.8460), 'population': (0., 2534.0630, 25.5314, 154.7233), 'PrevFireMask': (-1., 1., 0., 1.), 'FireMask': (-1., 1., 0., 1.)} 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import io
import imageio
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox

In [ ]:
def input_output_images(input_img:tf.Tensor, output_img:tf.Tensor, sample_size:int, num_in_channels:int, num_out_channels:int):
  combined = tf.concat([input_img, output_img], axis=2)
  combined = tf.image.random_crop(combined,[sample_size, sample_size, num_in_channels + num_out_channels])
  input_img = combined[:, :, 0:num_in_channels]
  output_img = combined[:, :, -num_out_channels:]
  return input_img, output_img
  #return combined
#input_output_images(input_img, output_img, sample_size, num_in_channels, 1)


In [ ]:

def random_crop_input_and_output_images(input_img:tf.Tensor, output_img:tf.Tensor, sample_size:int, num_in_channels:int, num_out_channels:int,) -> Tuple[tf.Tensor, tf.Tensor]:
  combined = tf.concat([input_img, output_img], axis=2)
  combined = tf.image.random_crop(
      combined,
      [sample_size, sample_size, num_in_channels + num_out_channels])
  input_img = combined[:, :, 0:num_in_channels]
  output_img = combined[:, :, -num_out_channels:]
  return input_img, output_img


def center_crop_input_and_output_images(input_img: tf.Tensor, output_img: tf.Tensor, sample_size: int,
) -> Tuple[tf.Tensor, tf.Tensor]:
  central_fraction = sample_size / input_img.shape[0]
  input_img = tf.image.central_crop(input_img, central_fraction)
  output_img = tf.image.central_crop(output_img, central_fraction)
  return input_img, output_img

In [ ]:

def _get_base_key(key: Text) -> Text:
  match = re.match(r'[a-zA-Z]+', key)
  if match:
    return match.group(1)
  raise ValueError(
      f'The provided key does not match the expected pattern: {key}')


def _clip_and_rescale(inputs: tf.Tensor, key: Text) -> tf.Tensor:
  base_key = _get_base_key(key)
  if base_key not in DATA_STATS:
    raise ValueError(
        'No data statistics available for the requested key: {}.'.format(key))
  min_val, max_val, _, _ = DATA_STATS[base_key]
  inputs = tf.clip_by_value(inputs, min_val, max_val)
  return tf.math.divide_no_nan((inputs - min_val), (max_val - min_val))


def _clip_and_normalize(inputs: tf.Tensor, key: Text) -> tf.Tensor:
  base_key = _get_base_key(key)
  if base_key not in DATA_STATS:
    raise ValueError(
        'No data statistics available for the requested key: {}.'.format(key))
  min_val, max_val, mean, std = DATA_STATS[base_key]
  inputs = tf.clip_by_value(inputs, min_val, max_val)
  inputs = inputs - mean
  return tf.math.divide_no_nan(inputs, std)

def _get_features_dict(
    sample_size: int,
    features: List[Text],
) -> Dict[Text, tf.io.FixedLenFeature]:
  sample_shape = [sample_size, sample_size]
  features = set(features)
  columns = [
      tf.io.FixedLenFeature(shape=sample_shape, dtype=tf.float32)
      for _ in features
  ]
  return dict(zip(features, columns))


def _parse_fn(example_proto: tf.train.Example, data_size: int, sample_size: int, num_in_channels: int, clip_and_normalize: bool, clip_and_rescale: bool, random_crop: bool, center_crop: bool,) -> Tuple[tf.Tensor, tf.Tensor]:
  if (random_crop and center_crop):
    raise ValueError('Cannot have both random_crop and center_crop be True')
  input_features, output_features = INPUT_FEATURES, OUTPUT_FEATURES
  feature_names = input_features + output_features
  features_dict = _get_features_dict(data_size, feature_names)
  features = tf.io.parse_single_example(example_proto, features_dict)

  if clip_and_normalize:
    inputs_list = [_clip_and_normalize(features.get(key), key) for key in input_features]
  elif clip_and_rescale:
    inputs_list = [_clip_and_rescale(features.get(key), key) for key in input_features]
  else:
    inputs_list = [features.get(key) for key in input_features]

  inputs_stacked = tf.stack(inputs_list, axis=0)
  input_img = tf.transpose(inputs_stacked, [1, 2, 0])

  outputs_list = [features.get(key) for key in output_features]
  assert outputs_list, 'outputs_list should not be empty'
  outputs_stacked = tf.stack(outputs_list, axis=0)

  outputs_stacked_shape = outputs_stacked.get_shape().as_list()
  assert len(outputs_stacked.shape) == 3, ('outputs_stacked should be rank 3''but dimensions of outputs_stacked'f' are {outputs_stacked_shape}')
  output_img = tf.transpose(outputs_stacked, [1, 2, 0])

  if random_crop:
    input_img, output_img = random_crop_input_and_output_images(input_img, output_img, sample_size, num_in_channels, 1)
  if center_crop:
    input_img, output_img = center_crop_input_and_output_images(input_img, output_img, sample_size)
  return input_img, output_img


def get_dataset(file_pattern: Text, data_size: int, sample_size: int, batch_size: int, num_in_channels: int, compression_type: Text, clip_and_normalize: bool, clip_and_rescale: bool, random_crop: bool, center_crop: bool) -> tf.data.Dataset:
  if (clip_and_normalize and clip_and_rescale):
    raise ValueError('Cannot have both normalize and rescale.')
  dataset = tf.data.Dataset.list_files(file_pattern)
  dataset = dataset.interleave(lambda x: tf.data.TFRecordDataset(x, compression_type=compression_type), num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  dataset = dataset.map(lambda x: _parse_fn(x, data_size, sample_size, num_in_channels, clip_and_normalize, clip_and_rescale, random_crop, center_crop),num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  return dataset

In [ ]:
dataset = get_dataset(
      file_pattern,
      data_size=64,
      sample_size=32,
      batch_size=100,
      num_in_channels=12,
      compression_type=None,
      clip_and_normalize=False,
      clip_and_rescale=False,
      random_crop=True,
      center_crop=False)

In [ ]:
features, firemask = next(iter(dataset))

In [ ]:
features.numpy()

array([[[[1.95000000e+02, 1.99936554e+02, 1.68170214e+00, ...,
          1.21779442e+00, 1.78264618e+01, 0.00000000e+00],
         [1.94000000e+02, 2.00140671e+02, 1.68601012e+00, ...,
          9.69509602e-01, 1.79482861e+01, 0.00000000e+00],
         [1.75000000e+02, 2.00350845e+02, 1.69042659e+00, ...,
          8.95979285e-01, 1.80741901e+01, 0.00000000e+00],
         ...,
         [1.18000000e+02, 2.02640366e+02, 1.78602302e+00, ...,
          1.89486527e+00, 2.08099117e+01, 0.00000000e+00],
         [1.13000000e+02, 2.02561890e+02, 1.78799415e+00, ...,
          1.82154536e+00, 2.08707848e+01, 0.00000000e+00],
         [1.01000000e+02, 2.02488281e+02, 1.78987932e+00, ...,
          5.15989423e-01, 2.09344978e+01, 0.00000000e+00]],

        [[1.98000000e+02, 2.00176468e+02, 1.68431592e+00, ...,
          8.52685094e-01, 1.78966389e+01, 0.00000000e+00],
         [1.89000000e+02, 2.00386230e+02, 1.68888044e+00, ...,
          8.95979404e-01, 1.80138321e+01, 0.00000000e+00],
        

In [ ]:
features_np = features.numpy()
firemask_np = firemask.numpy()

In [ ]:
type(features_np)

numpy.ndarray

In [ ]:
firemask.shape
firemask

<tf.Tensor: shape=(100, 32, 32, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
    

In [ ]:
np.concatenate((features_np, firemask_np), axis = 3, out = None, dtype = np.ndarray, casting = "same_kind")


array([[[[195.0, 199.93655395507812, 1.6817021369934082, ...,
          17.826461791992188, 0.0, 0.0],
         [194.0, 200.1406707763672, 1.6860101222991943, ...,
          17.948286056518555, 0.0, 0.0],
         [175.0, 200.35084533691406, 1.6904265880584717, ...,
          18.074190139770508, 0.0, 0.0],
         ...,
         [118.0, 202.64036560058594, 1.7860230207443237, ...,
          20.809911727905273, 0.0, 0.0],
         [113.0, 202.5618896484375, 1.787994146347046, ...,
          20.870784759521484, 0.0, 0.0],
         [101.0, 202.48828125, 1.789879322052002, ...,
          20.934497833251953, 0.0, 0.0]],

        [[198.0, 200.1764678955078, 1.6843159198760986, ...,
          17.896638870239258, 0.0, 0.0],
         [189.0, 200.38623046875, 1.688880443572998, ...,
          18.013832092285156, 0.0, 0.0],
         [181.0, 200.60089111328125, 1.6935627460479736, ...,
          18.13401222229004, 0.0, 0.0],
         ...,
         [112.0, 202.9178466796875, 1.7933461666107178, ...